This notebook steps through an example of using the efficient frontier to optmise a stock portfolio given price data 

Sources:
Article :     [Portfolio Optimization through mean-variance analysis](https://www.machinelearningplus.com/machine-learning/portfolio-optimization-python-example/)

In [2]:
#Import the libraries
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import math
from datetime import timedelta 
import operator
from functools import reduce
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting
import cvxpy as cp
from joblib import Parallel, delayed
import seaborn as sns
import plotly.offline
import DatabaseMainFnc as dmf
pd.set_option('display.max_columns', 38)

# Import libraries to fetch historical EUR/USD prices
from datetime import datetime,timedelta
from forex_python.converter import get_rate

In [3]:
dmf.gen_curr_csv()

'rates.csv'

In [5]:
rates_df = pd.read_csv('rates.csv')

In [6]:
rates_df=rates_df.set_index(pd.DatetimeIndex(rates_df['Date'].values))
rates_df.drop(columns=['Date'],axis=1, inplace=True)
rates_df.head()

,USD,JPY,GBP
2006-01-01,0.847673,0.007199,1.459215
2006-01-02,0.845594,0.007165,1.456664
2006-01-03,0.842105,0.007171,1.455075
2006-01-04,0.827609,0.007133,1.453172
2006-01-05,0.827267,0.007127,1.451800


In [11]:
#list(rates_df.columns)
input_curr = 'DOGE'

if not input_curr in list(rates_df.columns):
        return 'Currency not supported'


Currency not supported


In [15]:
input_curr = 'JPY'

rates_df[input_curr].values

array([0.00719942, 0.00716538, 0.00717103, 0.00713318, 0.00712708,
       0.00713216, 0.00713216, 0.00713216, 0.00725426, 0.00724008,
       0.00723014, 0.00727273, 0.00723956, 0.00723956, 0.00723956,
       0.00718236, 0.00718236, 0.00716795, 0.00719114, 0.00718649,
       0.00718649, 0.00718649, 0.00712504, 0.00711997, 0.00708617,
       0.0070497 , 0.00703284, 0.00703284, 0.00703284, 0.00704275,
       0.00703383, 0.00703136, 0.00699105, 0.00699056, 0.00699056,
       0.00699056, 0.00702839, 0.00708015, 0.00706664, 0.00704672,
       0.00710379, 0.00710379, 0.00710379, 0.00712504, 0.00716486,
       0.0071449 , 0.00713725, 0.00710379, 0.00710379, 0.00710379,
       0.00708667, 0.00706165, 0.00709673, 0.00714898, 0.00718856,
       0.00718856, 0.00718856, 0.00725847, 0.00723694, 0.00721709])

In [17]:
rates_df.head()

,USD,JPY,GBP
2006-01-01,0.847673,0.007199,1.459215
2006-01-02,0.845594,0.007165,1.456664
2006-01-03,0.842105,0.007171,1.455075
2006-01-04,0.827609,0.007133,1.453172
2006-01-05,0.827267,0.007127,1.451800


In [23]:
test_mul = pd.DataFrame()
test_mul['val'] = [-1, 0, 1, 0.5, 2]
test_mul['Date'] = rates_df.head().index

test_mul=test_mul.set_index(pd.DatetimeIndex(test_mul['Date'].values))
test_mul.drop(columns=['Date'],axis=1, inplace=True)

test_mul

,val
2006-01-01,-1.0
2006-01-02,0.0
2006-01-03,1.0
2006-01-04,0.5
2006-01-05,2.0


In [34]:
test_mul.values

array([[-1. ],
       [ 0. ],
       [ 1. ],
       [ 0.5],
       [ 2. ]])

In [45]:
my_df = rates_df.head()
my_df

,USD,JPY,GBP
2006-01-01,0.847673,0.007199,1.459215
2006-01-02,0.845594,0.007165,1.456664
2006-01-03,0.842105,0.007171,1.455075
2006-01-04,0.827609,0.007133,1.453172
2006-01-05,0.827267,0.007127,1.451800


In [46]:
np.array(test_mul) * np.array(my_df)

array([[-0.84767314, -0.00719942, -1.45921494],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.84210526,  0.00717103,  1.45507457],
       [ 0.41380452,  0.00356659,  0.72658577],
       [ 1.65453342,  0.01425415,  2.90360046]])

In [49]:
pd.DataFrame(np.array(test_mul) * np.array(my_df),columns=rates_df.columns,index=my_df.index)

,USD,JPY,GBP
2006-01-01,-0.847673,-0.007199,-1.459215
2006-01-02,0.000000,0.000000,0.000000
2006-01-03,0.842105,0.007171,1.455075
2006-01-04,0.413805,0.003567,0.726586
2006-01-05,1.654533,0.014254,2.903600


In [ ]:
start_date = datetime(2006,1,1).date()
print(start_date)
end_date = (datetime.today() - timedelta(1)).date()
print(end_date)

In [ ]:
# Generate list of dates
dates = []
for i in range((end_date - start_date).days + 1):
    dates.append((start_date + timedelta(i)))
dates[-10:]

In [ ]:
rates_df = pd.DataFrame()
rates_df['Date'] = dates
rates_df.head()

In [ ]:
# Generate list of dates
dates = []
for i in range(365 * num_years):
    dates.append((start + timedelta(i)).date())

In [ ]:
# Example call of FX rate API
t = datetime(2021, 3, 27)
get_rate("EUR", "USD", t)
# 1.1782

In [ ]:
# Load pre-generated rates csv, can skip rest of section (dev)
rates_df = pd.read_csv("rates.csv") 

In [ ]:
# Define time interval of FX rates
num_years = 1
end = datetime.today()
start = end - timedelta(365 * num_years)

In [ ]:
# Generate list of dates
dates = []
for i in range(365 * num_years):
    dates.append((start + timedelta(i)).date() + 1)

In [ ]:
dates[:10]

In [ ]:
rates_df = pd.DataFrame()
rates_df['Date'] = dates
rates_df.head()

In [ ]:
%time

rates_USD = []
for date in dates:
    rates_USD.append(get_rate('USD','EUR', date))
print(rates_USD[:10])

rates_JPY = []
for date in dates:
    rates_JPY.append(get_rate('JPY','EUR', date))
print(rates_JPY[:10])

rates_GBP = []
for date in dates:
    rates_GBP.append(get_rate('GBP','EUR', date))
print(rates_GBP[:10])

In [ ]:
rates[:10]

In [ ]:
rates_df['Rate'] = rates
rates_df.head()

In [ ]:
# Set dates as index
rates_df=rates_df.set_index(pd.DatetimeIndex(rates_df['Date'].values))
rates_df.drop(columns=['Date'],axis=1, inplace=True)
rates_df.head()

In [ ]:
# Saving down `rates_df` to speed up dev
#rates_df.to_csv('rates.csv',index = True)

In [ ]:
# Plot the daily price of USD in EUR
rates_df.plot(title='EUR/USD')
plt.show()

# [Portfolio Optimization through mean-variance analysis](https://www.machinelearningplus.com/machine-learning/portfolio-optimization-python-example/)

## 1. What is portfolio optimization?
Portfolio optimization is the process of creating a portfolio of assets, for which your investment has the maximum return and minimum risk.

## 2. What does a portfolio mean?
An investor’s portfolio basically is his/her investment in different kinds of assets from different companies.

For example, if you have investments in 3 companies, say, Google, Amazon and Tesla, then these 3 companies make up your investment portfolio.

But how do you invest in a company? You do so by purchasing assets of that company.

## 3. What are assets, returns and risk?
Assets are of various kinds. An asset is what you would purchase if you want to invest in a company.

These include, but are not limited to:
- Bonds
- Stocks
- Cash
- Real Estate
Usually when you build a portfolio, it is advisable to diversify your assets, or purchase different kinds of assets from different companies. For all assets, you will get a profit after a specified period of time. However, the profit may not be the same for each investment you make.

This profit is what we call returns.

For example, you will get returns from stocks when it’s market value goes up and similarly you will get returns from cash in form of interest.

But what if the company whose stocks you have purchased goes bankrupt?

This will lead to its stocks crashing in the share market and instead of gaining profits, you will also lose your capital investment.

This is what is called risk of investment.

Another aspect of risk is the fluctuations in the asset value. For certain assets, its value is highly volatile, that is, the value increases when the market goes up, and drops accordingly. Whereas certain other assets, like bonds and certain steady stocks, are relatively more resistant to market conditions, but may give lesser returns compared to high risk ones.

A good portfolio is one which gives us maximum return on our investment for minimum risk, as discussed earlier.

The next question is, how do we decide out of an infinite possible combinations for portfolios, the one which is optimum?

## 4. Modern Portfolio Theory (MPT)
Modern Portfolio Theory, or also known as mean-variance analysis is a mathematical process which allows the user to maximize returns for a given risk level.

It was formulated by H. Markowitz and while it is not the only optimization technique known, it is the most widely used.

MPT assumes that all investors are risk-averse, i.e, if there is a choice between low risk and high risk portfolios with the same returns, an investor will choose one with the low risk.

So, what is the MPT all about?

MPT encourages diversification of assets. It says that a high variance asset A if combined with diverse assets B and C, where A, B and C have little to no correlation, can give us a portfolio with low variance on returns.

This is the crux of the Modern Portfolio Theory.

## 5. What is Efficient Frontier?
We know every asset in a portfolio has its own rate expected returns and risks. It is possible to create multiple combinations of assets that can provide high returns for a pre-defined risk level.

Likewise, there can be multiple portfolios that give lowest risk for a pre-defined expected return.

Efficient frontier is a graph with ‘returns’ on the Y-axis and ‘volatility’ on the X-axis. It shows the set of optimal portfolios that offer the highest expected return for a given risk level or the lowest risk for a given level of expected return.

Portfolios that lie outside the efficient frontier are sub-optimal because they do not provide either enough return for the level of risk or have a higher risk for the defined rate of return.

We will revisit this with an example again.

Now that you understand the term of portfolio optimization, let’s see how its actually implemented.

## 6. Fundamental terms in portfolio optimization
There are some statistical terms required in optimization process without which an optimal portfolio can’t be defined. Don’t worry, I will simplify it and make it easy and clear.

We will go through each one through an example.

In this example, we are considering a portfolio made up of stocks from just 4 companies, Microsoft, Apple, Tesla and Facebook.

### Step 1: Pull the stock price data

We have this data from the data bases earlier in this script, so we can connect to our nasdaq database which has the closing price of company’s stock on the given day and pull this data for these stocks:

In [ ]:
%%time
df_Nasdaq=dmf.connectAndLoadDb('NASDAQ')

#set dates as index
df_Nasdaq=df_Nasdaq.set_index(pd.DatetimeIndex(df_Nasdaq['Date'].values))
df_Nasdaq.drop(columns=['Date'],axis=1, inplace=True)

#Stock list Tickers we want Microsoft, Apple, Tesla and Facebook
l_stocks=['AAPL', 'MSFT', 'TSLA', 'FB']

df_example=df_Nasdaq[l_stocks]
df_example.tail()

## Step 2: Calculate percentage change in stock prices

Next, we calculate the percentage change in stock prices of tesla everyday. You will notice that that we take the log of percentage change.

But take log?  The reason for this is that log of the returns is time additive.

That is, $ log(r_{13}) = log(r_{12}) + log(r_{23}) $

For example:,
If $ p_1 = 100,\quad p_2 = 110,\quad p_3 = 120  $
where $ p_1 $ is price of stock in time 1

Then:

$ log(r_{12}) = ln(\frac{p_2}{p_1}) = ln(\frac{110}{100}) = 9.53%, $

$ log(r_{23}) = ln(\frac{120}{110}) = 8.7% $ and

$ log(r_{13}) = log(r_{12}) + log(r_{23}) = 9.53 + 8.7 = 18.23%, $  which is same as $ ln(\frac{120}{100}) $.

This means a log change of +0.1 today and then -0.1 tomorrow will give you the same value of stock as yesterday. This is not true if you simply compute percentage change.

It is common practice in portfolio optimization to take log of returns for calculations of covariance and correlation.

In [ ]:
# Log of percentage change
tesla = df_example['TSLA'].pct_change().apply(lambda x: np.log(1+x))
tesla.tail()

## Variance
The variance in prices of stocks of Tesla are an important indicator of how volatile this investment will be (how returns can fluctuate).

It can be calculated for each company by using built in .var() function. Under the hood, the formula implemented by this function is given by:
$$ s^2 = \sum_{i=1}^N (x_i – \bar{x})^2 / N-1 $$

In [ ]:
# Variance
var_tesla = tesla.var()
var_tesla

Similarly for Facebook,

In [ ]:
# Log of Percentage change for Facebook
fb = df_example['FB'].pct_change().apply(lambda x: np.log(1+x))

# Variance
var_fb = fb.var()
var_fb

## Volatility
Volatility is measured as the standard deviation of a company’s stock.

If you carefully look at the formula for standard deviation, you will understand that it is just the square root of variance.

$$ s = \sqrt{ \sum_{i=1}^N (x_i – \bar{x})^2 / N-1} $$

But volatility for the annual standard deviation. What we get from square root of variance is the daily standard deviation. To convert it to annual standard deviation we multiply the variance by 250.

250 is used because there are 250 trading days in a year.

In [ ]:
# Volatility
tesla_vol = np.sqrt(var_tesla * 250)
fb_vol = np.sqrt(var_fb * 250)
tesla_vol, fb_vol

In [ ]:
# Volatility of our stocks plotted
df_example.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250)).plot(kind='bar')

## Covariance
Covariance measures the directional relationship between the returns on two assets.

A positive covariance means that returns of the two assets move together while a negative covariance means they move inversely. Risk and volatility can be reduced in a portfolio by pairing assets that have a negative covariance.

We can calculate the covariance of Tesla and Facebook by using the .cov() function.

You can notice that there is small positive covariance between Tesla and Facebook.

In [ ]:
# Log of Percentage change
test1 = df_example.pct_change().apply(lambda x: np.log(1+x))
# Covariance
test1['TSLA'].cov(test1['FB'])


## Correlation
Correlation, in the finance and investment industries, is a statistic that measures the degree to which two securities move in relation to each other. Correlations are used in advanced portfolio management, computed as the correlation coefficient, which has a value that must fall between -1.0 and +1.0.

You can think of correlation as a scaled version of covariance, where the values are restricted to lie between -1 and +1.

A correlation of -1 means negative relation, i.e, if correlation between Asset A and Asset B is -1, if Asset A increases, Asset B decreases.

A correlation of +1 means positive relation, i.e, if correlation between Asset A and Asset B is 1, if Asset A increases, Asset B increases.

A correlation of 0 means no relation, i.e, if correlation between Asset A and Asset B is 0, they dont have any effect on each other.

This is calculated using the .corr() function.

In line with the covariance, the correlation between Tesla and Facebook is also positive.

In [ ]:
test1['TSLA'].corr(test1['FB'])

## Expected Returns
Expected returns of an asset are simply the mean of percentage change in its stock prices. So, the value of expected return we obtain here are daily expected returns.

For a yearly expected return value, you will need to resample the data year-wise, as you will see further.

For expected returns, you need to define weights for the assets choosen.

In simpler terms, this means you need to decide what percentage of your total money to you want to hold in each company’s stock.

Usually this decision is done by using the optimization techniques we will discuss later but for now we will consider random weights.

First, let’s compute the log of percentage change.

In [ ]:
test2 = df_example.pct_change().apply(lambda x: np.log(1+x))
test2.head()

## Weights
Let’s define an array of random weights for the purpose of calculation. These weights will represent the percentage allocation of investments between these two stocks. They must add up to 1.

So, the problem of portfolio optimization is nothing but to find the optimal values of weights that maximizes expected returns while minimizing the risk (standard deviation).

In [ ]:
# Define weights for allocation
w = [0.2,0.2,0.3, 0.3]
e_r_ind = test2.mean()
e_r_ind

The total expected return for a portfolio is given by:

$$ E(R_p) = w_1E(R_1) + w_2E(R_2) + ….. w_nE(R_n)$$

Thus, $ e_r $, or total expected return can be calculated as:

In [ ]:
# Total expected return
e_r = (e_r_ind*w).sum()
e_r

### 7. Building an optimal risky portfolio
Now that you have gone through the building blocks of portfolio optimization, it is time to create an optimal portfolio using the same concepts.

We will be using stocks from 4 companies, namely, Microsoft, Apple, Tesla and Facebook for a period of 5 years.

You will learn to calculate the weights of assets for each one. Then, we will calculate the expected returns, minimum variance portfolio, optimal risky portfolio and efficient frontier. You will also learn a new term called Sharpe Ratio.

Let’s get started by pulling the required asset data from our database.

In [ ]:
%%time
df_Nasdaq=dmf.connectAndLoadDb('NASDAQ')

#set dates as index
df_Nasdaq=df_Nasdaq.set_index(pd.DatetimeIndex(df_Nasdaq['Date'].values))
df_Nasdaq.drop(columns=['Date'],axis=1, inplace=True)

#Stock list Tickers we want Microsoft, Apple, Tesla and Facebook
l_stocks=['AAPL', 'MSFT', 'TSLA', 'FB']

df_example=df_Nasdaq[l_stocks]
df_example=df_example['2015-01-01':'2020-01-01']
df_example.tail()

## 8. Covariance and Correlation matrix
The first step is to obtain a covariance and correlation matrix to understand how different assets behave with respect to each other. When we had a 2 asset portfolio, we directly plugged in the names of the assets into .cov() and .corr() functions.

In this case, we will need a matrix for better visualisation. This is also achieved by using the same 2 functions on our dataframe df.

Note that we perform necessary operations to display log change in prices of stocks each day.

In [ ]:
# Log of percentage change
cov_matrix = df_example.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

The covariance between Apple and Apple, or Tesla and Tesla is the variance of that asset.

The next step is to create the correlation matrix. Correlation ranges from -1 to 1.

Trivialy, an asset always has a perfectly positive correlation of 1 with itself.

In [ ]:
corr_matrix = df_example.pct_change().apply(lambda x: np.log(1+x)).corr()
corr_matrix

## Portfolio Variance
The formula for calculating portfolio variance differs from the usual formula of variance. It looks like this:

$$\sigma^2(Rp) = \sum{i=1}^{n} \sum_{j=1}^{n} w_i w_j COV(R_i, R_j) $$
Here, $ w_i $ and $ w_j $ denote weights of all assets from 1 to n (in our case from 1 to 4) and $ COV(R_i, R_j) $ is the covariance of the two assets denoted by i and j.
The simplest way to do this complex calculation is defining a list of weights and multiplying this list horizontally and vertically with our covariance matrix.

For this purpose, let’s define a random list of weights for all 4 assets. Remember that sum of weights should always be 1.

In [ ]:
# Randomly weighted portfolio's variance
w = {'AAPL': 0.1, 'MSFT': 0.2, 'TSLA': 0.5, 'FB': 0.2}
port_var = cov_matrix.mul(w, axis=0).mul(w, axis=1).sum().sum()
port_var

Thus we have found the portfolio variance. But for truly optimizing the portfolio, we cant plug in random weights. We will need to calculate it according to what gives us maximum expected returns.

### 9. Portfolio expected returns
The mean of returns (given by change in prices of asset stock prices) give us the expected returns of that asset.
The sum of all individual expected returns further multiplied by the weight of assets give us expected return for the portfolio.

Note that we use the resample() function to get yearly returns. The argument to function, ‘Y’, denotes yearly.
If we dont perform resampling, we will get daily returns, like you saw earlier in the ‘Fundamental Terms’ section.

In [ ]:
# Yearly returns for individual companies
ind_er = df_example.resample('Y').last().pct_change().mean()
ind_er

In [ ]:
# Portfolio expected returns
w = [0.1, 0.2, 0.5, 0.2]
port_er = (w*ind_er).sum()
port_er

## Plotting the efficient frontier
This is the aim of going through all the topics above, to plot the efficient frontier. Efficient frontier is a graph with ‘returns’ on the Y-axis and ‘volatility’ on the X-axis. It shows us the maximum return we can get for a set level of volatility, or conversely, the volatility that we need to accept for certain level of returns.

Below, you can see the calculations and code for finding the optimal weights of assets and plotting the efficient frontier for given portfolio.
But first, lets take a look at the volatiltilty and returns of individual assets for a better understanding.

In [ ]:
# Volatility is given by the annual standard deviation. We multiply by 250 because there are 250 trading days/year.
ann_sd = df_example.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))
ann_sd

In [ ]:
assets = pd.concat([ind_er, ann_sd], axis=1) # Creating a table for visualising returns and volatility of assets
assets.columns = ['Returns', 'Volatility']
assets

Tesla has the maximum risk attached but has low returns. Apple has low risk and high return rates. Facebook is somewhere in the middle.

Next, to plot the graph of efficient frontier, we need run a loop. In each iteration, the loop considers different weights for assets and calculates the return and volatility of that particular portfolio combination.

We run this loop a 10,000 times.

To get random numbers for weights, we use the np.random.random() function. But remember that the sum of weights must be 1, so we divide those weights by their cumulative sum.

In [ ]:
p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

num_assets = len(df_example.columns)
num_portfolios = 10000

In [ ]:
%%time
for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns = np.dot(weights, ind_er) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
    p_ret.append(returns)
    var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
    p_vol.append(ann_sd)

In [ ]:
data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(df_example.columns.tolist()):
    #print(counter, symbol)
    data[symbol+' weight'] = [w[counter] for w in p_weights]
    
portfolios  = pd.DataFrame(data)
portfolios.head() # Dataframe of the 10000 portfolios created    

You can see that there are a number of portfolios with different weights, returns and volatility. Plotting the returns and volatility from this dataframe will give us the efficient frontier for our portfolio.

In [ ]:
# Plot efficient frontier
portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.2, grid=True, figsize=[10,10])

### How to read the Efficient Frontier?
Each point on the line (left edge) represents an optimal portfolio of stocks that maximises the returns for any given level of risk.

The point (portfolios) in the interior are sub-optimal for a given risk level. For every interior point, there is another that offers higher returns for the same risk.

On this graph, you can also see the combination of weights that will give you all possible combinations:

Minimum volatility (left most point)
Maximum returns (top most point)
And everything in between.

In [ ]:
min_vol_port = portfolios.iloc[portfolios['Volatility'].idxmin()]
# idxmin() gives us the minimum value in the column specified.                               
min_vol_port

The minimum volatility is in a portfolio where the weights of Microsoft, Apple, Tesla and Facebook are 43%, 33%, 6% and 19% respectively. This point can be plotted on the efficient frontier graph as shown:

In [ ]:
# plotting the minimum volatility portfolio
plt.subplots(figsize=[10,10])
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)

The red star denotes the most efficient portfolio with minimum volatility.

It is worthwhile to note that any point to the right of efficient frontier boundary is a sup-optimal portfolio.

We found the portfolio with minimum volatility, but you will notice that the return on this portfolio is pretty low. Any sensible investor wants to maximize his return, even if it is a tradeoff with some level of risk.

The question arises that how do we find this optimal risky portfolio and finally optimize our portfolio to the maximum?

This is done by using a parameter called the **Sharpe Ratio**.

### Sharpe Ratio
The ratio is the average return earned in excess of the risk-free rate per unit of volatility or total risk. Volatility is a measure of the price fluctuations of an asset or portfolio.

The risk-free rate of return is the return on an investment with zero risk, meaning it’s the return investors could expect for taking no risk.

The optimal risky portfolio is the one with the highest Sharpe ratio. The formula for this ratio is:

$ R_p $ = return of the portfolio,$ \quad R_f =$ Risk free rate, $\quad 𝜎_p =$ standard deviation of the portfolio   

Sharpe Ratio = $ \frac{R_p - R_f}{𝜎_p} $   
Below is the code for finding out portfolio with maximum Sharpe Ratio. This portfolio is the optimized portfolio that we wanted to find. We define the risk-free rate to be 1% or 0.01.

### Optimal Risky Portfolio
An optimal risky portfolio can be considered as one that has highest Sharpe ratio.

In [ ]:
# Finding the optimal portfolio
rf = 0.01 # risk factor
optimal_risky_port = portfolios.iloc[((portfolios['Returns']-rf)/portfolios['Volatility']).idxmax()]
optimal_risky_port

You can notice that while the difference in risk between minimum volatility portfolio and optimal risky portfolio is just 1%, the difference in returns is 4%.
We can plot this point too on the graph of efficient frontier:
The green star represents the optimal risky portfolio.

In [ ]:
# Plotting optimal portfolio
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)

But how would have this portfolio actually perfomed this is something i want to expand on:

In [ ]:
df_example=df_Nasdaq[l_stocks]
df_example=df_example['2020-01-01':'2021-01-04']
df_example.tail()

In [ ]:
# Weights as advised above returns
w = [0.474041, 0.515592, 0.002323, 0.008043]

#actual log of pct change
actual_performance =df_example.pct_change().apply(lambda x: np.log(1+x)).sum()

#now we have summed we inverse the log & use the advised weights & Sum to get the portfolios performnce
port_performance=((actual_performance.apply(lambda x: math.exp(x)-1))*w).sum()

print("Expected Return: ",round(optimal_risky_port[0],2))
print("Actual Return:   ",round(port_performance,2))
pd.DataFrame([optimal_risky_port[0],((actual_performance.apply(lambda x: math.exp(x)-1))*w).sum()],index=['Expected Return','Actual Return']).plot(kind='bar')


So for 2020 our optimal portfolio based off the 2015 - 2019 data outperformed our expected returns by **21% !!!!!!!!!!!!!!**.

# Using this approach at scale:
We need to use a library thats built for this to do this at scale at speed: [PyPortfolioOpt](https://pyportfolioopt.readthedocs.io/en/latest/index.html)



In [ ]:
df_example=df_Nasdaq[l_stocks]
df_example=df_example['2015-01-01':'2020-01-01']
df_example.plot()

In [ ]:
%%time
#Calculate expected annulised returns & annual sample covariance matrix of the daily asset
example_mu = expected_returns.mean_historical_return(df_example)
example_S = risk_models.sample_cov(df_example)
print(example_mu)
example_S

In [ ]:
%%time
# Optomise for maximal Sharpe ratio
example_ef= EfficientFrontier(example_mu, example_S) #Create the Efficient Frontier Object
example_weights = example_ef.max_sharpe()
example_cl_weights= example_ef.clean_weights()
print(example_cl_weights)
print('')
example_ef.portfolio_performance(verbose=True)

In [ ]:
ef = EfficientFrontier(example_mu, example_S, weight_bounds=(None, None))


fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)

# Find the tangency portfolio
ef.max_sharpe()
ret_tangent, std_tangent, _ = ef.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

# Generate random portfolios
n_samples = 10000
w = np.random.dirichlet(np.ones(len(example_mu)), n_samples)
rets = w.dot(example_mu)
stds = np.sqrt(np.diag(w @ example_S @ w.T))
sharpes = rets / stds
ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

# Output
ax.set_title("Efficient Frontier with random portfolios")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

portfolio_val = 1000
latest_prices=get_latest_prices(df_example)
weights=example_cl_weights
da=DiscreteAllocation(weights,latest_prices,total_portfolio_value=portfolio_val)
allocation, leftover = da.lp_portfolio()

print('Discrete allocation  :', allocation)
print('Funds Remaining:', leftover)